# Introducing the LabTrinket

This notebook should help you get started using LabTrinket. As well as the Trinket M0, you will also need a 220 Ohm resistor and ideally a photoresistor or other resistive sensor to perform some of the tests in this notebook. Ideally your Trinket should also be mounted on a breadboard.
The LabTrinket is based on the Trinket M0 microcontroller from Adafruit. It is designed to use the Tinket as a low cost component for take-home science labs allowing data gathering and some control of experiments. The Trinket is run connected to a host computer and responds to simple serial commands over USB. LabTrinket consists of two parts. 
1) A Circuit Python script to place on the Trinket microcontroller to interpret and execute serial commands from the host computer
2) A python library for the host computer to allow it to communicate with the Trinket
## Before using the Trinket
To use the trinket you need to connect it via a USB cable to your computer. It is compatible with Windows, Linux and Mac computers.  It should be a standard USB to micro USB as used to connect most smartphone but make sure it is not a "charging only" cable. 

## Setting up the First Time

### Windows Drivers

For Windows you will need to download and install drivers from here: https://learn.adafruit.com/adafruit-trinket-m0-circuitpython-arduino/windows-driver-installation  This step is not needed for Linux or MacOS. Follow the instructions at that link.

### Circuit Python

Connect the Board via the USB. You should see some LEDs flash on the board and you will notice the Trinket appears as an external drive on your computer, like a flash drive. 

The Trinket M0 uses an onboard Python interpreter called CircuitPython. Before first use you will need to update to the latest version. Visit this section: https://learn.adafruit.com/adafruit-trinket-m0-circuitpython-arduino?view=all#what-is-circuitpython and follow the instructions _very carefully_ to install the latest version on the Trinket.

### LabTrinket Code

The LabTrinket client code is called "main.py" can be found in the "CircuitPython" folder of this repo. https://github.com/jhoyland/LabTrinket 
After you have updated CircuitPython and restarted your Trinket (by plugging it out and in again) you can copy main.py onto it. It will overwrite the main.py which is already on your Trinket. 

### Install LED Libraries
The Circuit Python on the Trinket needs a couple of libraries to operate the on board LED. You can get hold of the complete bundle of libraries from https://github.com/adafruit/Adafruit_CircuitPython_Bundle/releases/tag/20200925 You probably installed CircuitPython 5 in the earlier step so get version 5 of the bundle. Unzip this somewhere convenient. 
From the bundle you need two libraries:
```
adafruit_dotstar.mpy
adafruit_pypixelbuf.mpy
```
On your Trinket there should be a folder called \lib  (you can create one if it's not there)  Copy the two libraries into the lib folder on the Trinket. That's the whole install for the Trinket itself. You don't need the rest of the bundle you just unzipped and it can be deleted if you want to save space.

### Install PySerial

You need one last bit of install. The base Anaconda installation does not include the serial communications package. To install, you need to start up an "Ananconda Prompt" - on Windows this is on the Start Menu under "Anaconda". There you can type: ```conda install -c anaconda pyserial```
Follow the on screen instructions. This should install the latest version. If not using Anaconda use pip or equivalent to install pyserial

## Test your Trinket!

With this all done you should now be able to test your Trinket. Make sure it is plugged in and mounted on a protoboard. This notebook will walk you through how to use the device. First thing to do is to import the labtrinket library and the serial library


In [1]:
import labtrinket
import serial

### Set up your serial connection
The labtrinket code communicates with the trinket through a serial connection. When you plug in your Trinket, your computer will assign it a serial port. For my Windows computer it was 'COM17' yours will probably be different. If you are running Linux or Mac you should also be able to find the name of the port used. There it will not be named COM but something else. Run the commands below changing the port variable to match your computer.

In [2]:
ser = serial.Serial()
ser.port = 'COM11'
ser.baudrate = 9600
ser.open()

Now you create a LabTrinket object and associate it with the serial connection you just made

In [3]:
trinket = labtrinket.LabTrinket(ser)

### Control the onboard LED
The triket has an onboard LED. This is actually 3 LEDs in one - red, green and blue - which you can control. Try setting a color.

In [5]:
trinket.ledSetColor(red=200,green=90,blue=0)

You can set the red, green and blue individually sending values between 0 and 255. If you don't specify one of the colors then it won't change. Try re-running the previous cell with different values.

You can also change the overall brightness with the ledSetBrightness command. This takes values from 0-100. 

In [8]:
trinket.ledSetBrightness(20)

You can turn the led off by either changing all the red, green and blue to 0 or setting the brightness to 0. Or by using the ledOff command.

In [9]:
trinket.ledOff()

The advantage of the ledOff is that when you turn it back on it will remember the color setting.

In [10]:
trinket.ledOn()

### Control the ADC
The Analog to Digital Converter (ADC) allows you to read voltages. We will use this in many experiments. It means you can essentially use your Trinket as voltmeter. 
*Warning: The Trinket can only handle voltages of between 0 and 3.3V. Be careful to never connect voltages outside this range to the inputs*
Lets try and measure something. On the same board as your Trinket connect your photoresistor and a 220$\Omega$ resistor in series. Connect the free leg of the photo resistor the 3V output on the Trinket (the pin furthest from the USB on the bottom).
Connect the free leg of the other resistor to the Ground pin (top row, second pin in from the USB). Finally connect  a wire between the point where the two resistors connect and the pin marked '0'. Now we can try and measure some light.

In [11]:
trinket.adcRequestValue()

The line above requests an ADC value from the trinket. It is not necessarily sent immediately. You can check though if the value is ready by writing:

In [12]:
trinket.adcGetValue()

True

If this returns True the value is ready. The function above will alredy have read it and stored it in the 'value' of the trinket object. You can see the value here. By the way the values are the raw ADC values. 

In [13]:
print(trinket.value)

44944


You can also collect a bunch of values. The adcRun command starts collecting values at regular intervals (the default is 1 second). The trinket starts sending the values automatically but you need to read them one at a time. Start the ADC running:

In [14]:
trinket.adcRun()

With this code we will read 50 values and print them out along with an index number. Note that we are using adcGetValue again in an if statement, so it will keep trying until it gets each value. _Note_: sometimes the code can hang up here. If it does unplug the Trinket and plug it back in again, then restart the kernal on this notebook and run the lines again to this point.

In [13]:
i=0
while i<50:
    if trinket.adcGetValue():
        print("{}: {}".format(i,trinket.value))
        i = i+1


0: 46688
1: 46608
2: 46464
3: 45952
4: 45680
5: 45472
6: 45760
7: 45984
8: 46272
9: 46496
10: 46560
11: 46640
12: 46400
13: 46160
14: 45680
15: 45504
16: 45584
17: 45904
18: 46096
19: 46240
20: 46512
21: 46592
22: 46560
23: 46400
24: 46176
25: 45728
26: 45552
27: 45472
28: 45696
29: 45920
30: 46128
31: 46240
32: 46512
33: 46544
34: 46672
35: 46528
36: 46400
37: 46032
38: 45728
39: 45520
40: 45504
41: 45616
42: 45776
43: 45824
44: 46064
45: 46368
46: 46544
47: 46672
48: 46592
49: 46512


Now stop the ADC running:

In [15]:
trinket.adcStop()

### Control the DAC
The Trinket can also act as a voltage source through its Digital to Analogue Converter (DAC). This is on the pin marked '1'. Disconnect the photoresistor circuit you made and now connect an LED in series with a 220$\Omega$ resistor. Connect the positive leg of the LED to pin '1' and the free leg of the resistor to GND.
Turn on the DAC:

In [ ]:
trinket.dacOn()

Most likely nothing will happen! LEDs require a  minimum votage to light up change the voltage to 3 to see it light up.

In [ ]:
trinket.dacVolts(3.2)

If you still can't see any light, check your LED is the right way round! The positive end should be the long leg. Once its working try rerunning the live above with different voltages to see where the cutoff is for the LED. It won't let you provide a voltage outside the 0-3.3V range btw. Once you are done you can turn of the DAC.

In [ ]:
trinket.dacOff()

Finally close the Serial connection. By the way if communication is interrupted while you are running these (e.g. the USB gets unplugged) you may find you have to restart the trinket and the Python kernel of this notebook.

In [16]:
ser.close()

## Conclusion
We will use the Trinket extensively so make sure you are comfortable with all these commands, you can use this notebook as a reference later on. Feel free to play further with the notebook.